In [ ]:
import warnings
warnings.filterwarnings('ignore')
import glob
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier

from sklearn.preprocessing import OneHotEncoder
import random
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
!pip install Bayesian-Optimization

In [ ]:
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score


In [ ]:
import os
os.chdir('/content/drive/MyDrive/data/dacon/신용카드 사용자 연체 예측 AI')
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
submit=pd.read_csv('sample_submission.csv')


In [ ]:
# 결측치 합
train.isnull().sum()

In [ ]:
# 결측치 퍼센트 
missing_df = train.isnull().sum().reset_index()
missing_df.columns = ['column', 'count']
missing_df['ratio'] = missing_df['count'] / train.shape[0]
missing_df.loc[missing_df['ratio'] != 0]

In [ ]:
# 기본적인 데이터 
train.describe()

In [ ]:
object_col = []
int_col = []
for col in train.columns:
    if train[col].dtype == 'object':
        object_col.append(col)
    else:
        int_col.append(col)

In [ ]:
# 범주형
object_col

In [ ]:
# 연속형
int_col

In [ ]:
# 종속변수 체크
train['credit'].value_counts().plot(kind='bar') 

In [ ]:
# 범주형 변수 분포
for col in object_col: 
    train[col].value_counts().plot(kind='bar') 
    plt.title(col) 
    plt.show()

In [ ]:
# 이산형 변수 분포
for col in int_col:
    sns.distplot(train.loc[train[col].notnull(), col])
    plt.title(col)
    plt.show()

In [ ]:
# 이산형 변수 EDA 
train.loc[:,int_col].describe()

In [ ]:
numerical_feature = list(set(train.columns) - set(object_col) - set(['credit']))
numerical_feature = np.sort(numerical_feature)
numerical_feature


In [ ]:
train.loc[:,int_col]

In [ ]:
# # 이변수, 삼변수 탐색
# sns.pairplot(train[list(numerical_feature) + ['credit']], hue='credit', 
#              x_vars=numerical_feature, y_vars=numerical_feature)
# plt.show()

In [ ]:
# # 수치형, 명목형 변수 간의 관계 탐색
# unique_list = train['credit'].unique()
# for row in object_col:
#     for col in numerical_feature:
#         plt.figure(figsize=(12,6))
#         sns.boxplot(x=row, y=col, hue='credit', data=train.dropna())
#         plt.title(row + " - {}".format(col))
#         plt.show()



In [ ]:
train.loc[:,int_col]

In [ ]:

train = train.drop(['index'], axis=1)
train.fillna('NAN', inplace=True) 


test = test.drop(['index'], axis=1)
test.fillna('NAN', inplace=True)

In [ ]:
# MinMaxScaler
# from sklearn import preprocessing as pc

# for i in int_col:
#     if i == 'credit':
#         continue
#     train[[i]] =pc.MinMaxScaler((0,5)).fit_transform(train[[i]])
# # titanic[['Fare']] = pc.MinMaxScaler((0,10)).fit_transform(titanic[['Fare']])

In [ ]:
train.loc[:,object_col]

In [ ]:
train

In [ ]:
# OnehotEncoder 부분
enc = OneHotEncoder()
enc.fit(train.loc[:,object_col])


train_onehot_df = pd.DataFrame(enc.transform(train.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
train.drop(object_col, axis=1, inplace=True)
train = pd.concat([train, train_onehot_df], axis=1)

test_onehot_df = pd.DataFrame(enc.transform(test.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
test.drop(object_col, axis=1, inplace=True)
test = pd.concat([test, test_onehot_df], axis=1)

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in skf.split(train, train['credit']):
    folds.append((train_idx, valid_idx))

In [ ]:
# 데이터 분리는 StratifiedKFold 를 사용하여 y값 분포를 비슷하게 분리시킴. -> 5-fold
# lightgbm의 default parameter로 훈련.
# 30번 이상 개선 없을 경우 중단.
# 각 5개의 fold를 훈련하여 저장

random.seed(42)
lgb_models={}
for fold in range(5):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    X_train, X_valid, y_train, y_valid = train.drop(['credit'],axis=1).iloc[train_idx].values, train.drop(['credit'],axis=1).iloc[valid_idx].values,\
                                         train['credit'][train_idx].values, train['credit'][valid_idx].values 
    lgb = LGBMClassifier(n_estimators=1000)
    lgb.fit(X_train, y_train, 
            eval_set=[(X_train, y_train), (X_valid, y_valid)], 
            early_stopping_rounds=30,
           verbose=100)
    lgb_models[fold]=lgb
    print(f'================================================================================\n\n')


In [ ]:
# 데이터 분리는 StratifiedKFold 를 사용하여 y값 분포를 비슷하게 분리시킴. -> 5-fold
# lightgbm의 default parameter로 훈련.
# 30번 이상 개선 없을 경우 중단.
# 각 5개의 fold를 훈련하여 저장

random.seed(42)
xgb_models={}
for fold in range(5):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    X_train, X_valid, y_train, y_valid = train.drop(['credit'],axis=1).iloc[train_idx].values, train.drop(['credit'],axis=1).iloc[valid_idx].values,\
                                         train['credit'][train_idx].values, train['credit'][valid_idx].values 
    xgb = XGBClassifier(n_estimators=100)
    xgb.fit(X_train, y_train, 
            eval_set=[(X_train, y_train), (X_valid, y_valid)], 
            # early_stopping_rounds=30,
           verbose=100
            )
    xgb_models[fold]=xgb
    print(f'================================================================================\n\n')


In [ ]:
# 데이터 분리는 StratifiedKFold 를 사용하여 y값 분포를 비슷하게 분리시킴. -> 5-fold
# lightgbm의 default parameter로 훈련.
# 30번 이상 개선 없을 경우 중단.
# 각 5개의 fold를 훈련하여 저장

random.seed(42)
xgb_models={}
for fold in range(5):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    X_train, X_valid, y_train, y_valid = train.drop(['credit'],axis=1).iloc[train_idx].values, train.drop(['credit'],axis=1).iloc[valid_idx].values,\
                                         train['credit'][train_idx].values, train['credit'][valid_idx].values 
    xgb = XGBClassifier(n_estimators=100)
    xgb.fit(X_train, y_train, 
            eval_set=[(X_train, y_train), (X_valid, y_valid)], 
            # early_stopping_rounds=30,
           verbose=100
            )
    xgb_models[fold]=xgb
    print(f'================================================================================\n\n')


In [ ]:
  from sklearn.metrics import r2_score, mean_squared_error
  import xgboost as xgb

  # MAPE Metric
  def mean_absolute_percentage_error(y_test, y_pred):
      y_test, y_pred = np.array(y_test), np.array(y_pred)
      return np.mean(np.abs((y_test - y_pred) / y_test)) * 100

  # 탐색 대상 함수 (XGBRegressor)
  def XGB_cv(max_depth,learning_rate, n_estimators, gamma
             ,min_child_weight, subsample
             ,colsample_bytree, silent=True, nthread=-1):

      # 모델 정의
      model = xgb.XGBRegressor(max_depth=int(max_depth),
                                learning_rate=learning_rate,
                                n_estimators=int(n_estimators),
                                gamma=gamma,
                                min_child_weight=min_child_weight,
                                subsample=subsample,
                                colsample_bytree=colsample_bytree, 
                                nthread=nthread
                                )
      # 모델 훈련
      model.fit(X_train, y_train)

      # 예측값 출력
      y_pred= model.predict(X_valid)

      # 각종 metric 계산
      rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
      r2 = r2_score(y_valid, y_pred)
      mape = mean_absolute_percentage_error(y_valid, y_pred)

      # 오차 최적화로 사용할 metric 반환
      return r2

In [ ]:
  #  bayesian-optimization 라이브러리의 BayesianOptimization 클래스 import
  from bayes_opt import BayesianOptimization
  import numpy as np

  # 실험해보고자하는 hyperparameter 집합
  pbounds = {'max_depth': (3, 7),
                'learning_rate': (0.01, 0.2),
                'n_estimators': (5000, 10000),
                'gamma': (0, 100),
                'min_child_weight': (0, 3),
                'subsample': (0.5, 1),
                'colsample_bytree' :(0.2, 1)
                }

  # Bayesian optimization 객체 생성
  # f : 탐색 대상 함수, pbounds : hyperparameter 집합
  # verbose = 2 항상 출력, verbose = 1 최댓값일 때 출력, verbose = 0 출력 안함
  # random_state : Bayesian Optimization 상의 랜덤성이 존재하는 부분을 통제 
  bo=BayesianOptimization(f=XGB_cv, pbounds=pbounds, verbose=2, random_state=1 )    

  # 메소드를 이용해 최대화 과정 수행
  # init_points :  초기 Random Search 갯수
  # n_iter : 반복 횟수 (몇개의 입력값-함숫값 점들을 확인할지! 많을 수록 정확한 값을 얻을 수 있다.)
  # acq : Acquisition Function들 중 Expected Improvement(EI) 를 사용
  # xi : exploration 강도 (기본값은 0.0)
  bo.maximize(init_points=2, n_iter=10, acq='ei', xi=0.01)

  # ‘iter’는 반복 회차, ‘target’은 목적 함수의 값, 나머지는 입력값을 나타냅니다. 
  # 현재 회차 이전까지 조사된 함숫값들과 비교하여, 현재 회차에 최댓값이 얻어진 경우, 
  # bayesian-optimization 라이브러리는 이를 자동으로 다른 색 글자로 표시하는 것을 확인할 수 있습니다

  # 찾은 파라미터 값 확인
  print(bo.max)


In [ ]:
print(bo.max)

In [ ]:
bo.max['params']

In [ ]:
# for i in bo.max['params'].keys()
for key, val in bo.max['params'].items():
    print(key)
    print(type(val))
# bo.max.items()[0]

In [ ]:
from sklearn.model_selection import GridSearchCV
grid_dtree = GridSearchCV(XGBClassifier(), param_grid=bo.max['params'], cv=3, refit=True)

In [ ]:
grid_cv = GridSearchCV(dt, param_grid=bo.max['params'], scoring='accuracy', cv=skfold)
best_df_clf = grid_cv.best_estimator_
pred1 = best_df_clf.predict(X_test)

In [ ]:
# 데이터 분리는 StratifiedKFold 를 사용하여 y값 분포를 비슷하게 분리시킴. -> 5-fold
# lightgbm의 default parameter로 훈련.
# 30번 이상 개선 없을 경우 중단.
# 각 5개의 fold를 훈련하여 저장

random.seed(42)
xgb_models={}
for fold in range(5):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    X_train, X_valid, y_train, y_valid = train.drop(['credit'],axis=1).iloc[train_idx].values, train.drop(['credit'],axis=1).iloc[valid_idx].values,\
                                         train['credit'][train_idx].values, train['credit'][valid_idx].values 
    xgb = XGBClassifier(
                        # n_estimators=100,
                        colsample_bytree=0.2,
                        gamma= 0.0,
                        learning_rate= 0.01,
                        max_depth= 7,
                        min_child_weight = 3.0,
                        n_estimators= 7499,
                        subsample= 0.5)
    xgb.fit(X_train, y_train, 
            eval_set=[(X_train, y_train), (X_valid, y_valid)], 
            # early_stopping_rounds=30,
           verbose=100
            )
    xgb_models[fold]=xgb
    print(f'================================================================================\n\n')


In [ ]:
    lgb.fit(X_train, y_train, 
            eval_set=[(X_train, y_train), (X_valid, y_valid)], 
            early_stopping_rounds=30,
           verbose=100)

In [ ]:
    # lgb.fit(X_train, y_train, 
    #         eval_set=[(X_train, y_train), (X_valid, y_valid)], 
    #         early_stopping_rounds=30,
    #        verbose=100)

In [ ]:
def bayesOpt(train_x, train_y):
    lgbBO = BayesianOptimization(lgb_evaluate, {  'numLeaves':  (5, 90),  'maxDepth': (2, 90),   'scaleWeight': (1, 10000),  'minChildWeight': (0.01, 70), 'subsample': (0.4, 1), 'colSam': (0.4, 1) })
    lgbBO.maximize(init_points=5, n_iter=30)
    print(lgbBO.res)
    return lgbBO

In [ ]:
def lgb_evaluate(numLeaves, maxDepth, scaleWeight, minChildWeight, subsample, colSam, output = 'score'):
    reg=LGBMClassifier(num_leaves=31, max_depth= 2,scale_pos_weight= scaleWeight, min_child_weight= minChildWeight, subsample= 0.4, colsample_bytree= 0.4, learning_rate=0.05,   n_estimators=20)
    scores = cross_val_score(reg, X_train, y_train, cv=5, scoring='roc_auc')
    # scores = cross_val_score(reg, train_x, train_y, cv=5, scoring='neg_mean_squared_error')
 
    if output == 'score' :
      return np.mean(scores)
    if output == 'model' :
      return reg

In [ ]:
y_train

In [ ]:
lgbBO = bayesOpt(X_train, y_train)

In [ ]:
submit.iloc[:,1:]=0
for fold in range(5):
    submit.iloc[:,1:] += lgb_models[fold].predict_proba(test)/5

In [ ]:
submit.iloc[:,1:]=0
for fold in range(5):
    submit.iloc[:,1:] += xgb_models[fold].predict_proba(test)/5

In [ ]:
submit

In [ ]:
submit.to_csv('baseline_submission2.csv', index=False) # 0.7272812144
